In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Low Resource

/content/drive/MyDrive/Low Resource


In [ ]:
!pip install openai pylcs -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import openai
import os
import json
import glob
import time
import pylcs
import openai.error
import uuid
import csv
import glob
import openai.error
import helpers
import difflib
import re

In [ ]:
openai.api_key = 'INSERT KEY HERE'

In [ ]:
with open('COVIDQA_TWO_SHOT_CYCLE_CONSISTENCY_REDO.json','r') as f:
  dataset = json.load(f)

In [ ]:
def find_answer_context(context, answer, max_context_length=500, num_sentences_before=1, num_sentences_after=1):
    # Strip leading and trailing whitespaces from the answer
    answer = answer.strip()

    # Find the sentence that contains the answer
    context_sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', context)
    target_sentence_index = next((i for i, s in enumerate(context_sentences) if answer.lower() in s.lower()), None)

    # If no sentence contains the answer, return None
    if target_sentence_index is None:
        return None

    # Determine the start and end indices of the context window
    context_start = max(0, target_sentence_index - num_sentences_before)
    context_end = min(len(context_sentences), target_sentence_index + num_sentences_after + 1)

    # Extract the answer-containing context
    answer_context = ' '.join(context_sentences[context_start:context_end])

    return answer_context

def answer_processing(input_string):
  array_match = re.search(r"Answers:\s+(\[.*\])", input_string)

  if array_match:
      array_text = array_match.group(1)
      try:
        answers_array = eval(array_text)  # Convert the string to a list
      except:
        return []

      return answers_array
  else:
      print("No answers array found in the input string.")

  return []

In [ ]:
def make_openai_request(prompt: str, model: str, max_tokens=8000):
    max_retry_count = 10
    retry_count = 1
    retry_time_sec = 20

    while retry_count < max_retry_count:
        try:

            response = openai.ChatCompletion.create(
                model=model, messages=prompt)
            answer = response["choices"][0]["message"]["content"]

            # answer = answer.strip().strip('"')
            return answer
        except openai.error.APIConnectionError as e:
            print(
                f"API connection error: {e}.\nRetrying in {retry_time_sec}...")
            retry_count += 1
            time.sleep(retry_time_sec)
        except openai.error.RateLimitError as e:
            # print(
                # f"Rate limit error: {e}.\nRetrying in {retry_time_sec}...")
            retry_count += 1
            time.sleep(retry_time_sec)
        except openai.error.Timeout as e:
            print(f"Timeout error: {e}.\nRetrying in {retry_time_sec}...")
            retry_count += 1
            time.sleep(retry_time_sec)
        except openai.error.ServiceUnavailableError as e:
            print(
                f"ServiceUnavailable error: {e}.\nRetrying in {retry_time_sec}...")
            retry_count += 1
            time.sleep(retry_time_sec)
        except OSError as e:
          from google.colab import drive
          drive.mount('/content/drive')
          %cd drive/MyDrive/Low Resource
        except openai.error.OpenAIError as e:
            print(f"Unhandled OpenAI error occurred: {e}")

        except Exception as e:
            print(f"Unhandled error occurred: {e}")
            break

        if retry_count >= max_retry_count:
            print(
                f"Retries of {retry_count} exceeded max_retry_count of {max_retry_count}")

In [ ]:
starting = 338
total_size = len(dataset["data"])
curr_file = "COVIDQA_TWO_SHOT_CYCLE_CONSISTENCY"

In [ ]:
example_context = dataset["data"][0]["paragraphs"][0]["context"]
example_question = dataset["data"][0]["paragraphs"][0]["qas"][0]["question"]
example_answer = dataset["data"][0]["paragraphs"][0]["qas"][0]["answers"][0]["text"]
example_context = find_answer_context(example_context, example_answer)

In [ ]:
for i in range(starting, total_size):
  curr_context = dataset["data"][i]["paragraphs"][0]["context"]
  rem_idx = []
  changed = 0
  for j in range(len(dataset["data"][i]["paragraphs"][0]["qas"])):
    curr_question = dataset["data"][i]["paragraphs"][0]["qas"][j]["question"]
    curr_answer = dataset["data"][i]["paragraphs"][0]["qas"][j]["answers"][0]["text"]
    short_context = find_answer_context(curr_context, curr_answer)

    cycle_prompt = [
                  {"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": f"Using only the given context, answer the given questions using only substrings from the context. If the answer cannot be found return the empty string. Do not generate any text or alter any capitalization or punctuation. Context: {example_context}. Question: {example_question}."},
                  {"role": "assistant", "content": f"Answer: {example_answer}"},
                  {"role": "user", "content": f"Using only the given context, answer the given questions using only substrings from the context. If the answer cannot be found return the empty string. Do not generate any text or alter any capitalization or punctuation. Context: {short_context}. Question: {curr_question}."}
                  ]
    gen_answer = make_openai_request(cycle_prompt, "gpt-4")
    time.sleep(1)
    lcs = helpers.lcs(gen_answer, curr_answer)
    if len(lcs) > 3:
      continue
    else:
      rem_idx.append(j)

  rem_idx.sort(reverse=True)
  for idx in rem_idx:
    changed += 1
    del dataset["data"][i]["paragraphs"][0]["qas"][idx]

  try:
      with open(f"{curr_file}.json", "w") as f:
        json.dump(dataset, f)
        print(f"Done with passage {i}/{total_size} and changed {changed}")
        print("    -------> Saved to Json")
  except OSError as e:
        from google.colab import drive
        drive.mount('/content/drive')
        %cd drive/MyDrive/Low Resource

Done with passage 338/465 and changed 0
    -------> Saved to Json
Done with passage 339/465 and changed 0
    -------> Saved to Json
Done with passage 340/465 and changed 0
    -------> Saved to Json
Done with passage 341/465 and changed 4
    -------> Saved to Json
Done with passage 342/465 and changed 0
    -------> Saved to Json
Done with passage 343/465 and changed 1
    -------> Saved to Json
Done with passage 344/465 and changed 0
    -------> Saved to Json
Done with passage 345/465 and changed 0
    -------> Saved to Json
Done with passage 346/465 and changed 1
    -------> Saved to Json
Done with passage 347/465 and changed 0
    -------> Saved to Json
Done with passage 348/465 and changed 0
    -------> Saved to Json
Done with passage 349/465 and changed 1
    -------> Saved to Json
Done with passage 350/465 and changed 1
    -------> Saved to Json
Done with passage 351/465 and changed 0
    -------> Saved to Json
Done with passage 352/465 and changed 0
    -------> Saved to 